In [1]:
%load_ext autoreload
%autoreload 2

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import time

from enum import Enum, auto
from collections import namedtuple
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit import Parameter as qiskit_Parameter
from qiskit.circuit import Gate as qiskit_Gate
from qiskit.circuit.library import RXGate, RYGate, RZGate, CZGate
from qiskit_aer import AerSimulator

In [12]:
np.set_printoptions(precision=3, floatmode='fixed', suppress=True)

In [24]:
class InitializeMode(Enum):
    ZERO = auto()
    RANDOM = auto()

class Parameter:
    
    MIN_VALUE = 0.
    MAX_VALUE = np.pi * 2
    VALUE_RANGE = (MIN_VALUE, MAX_VALUE)
    INIT_MODE_ZERO = InitializeMode.ZERO
    INIT_MODE_RANDOM = InitializeMode.RANDOM
    
    def __init__(self, name, init_value=None):
        init_value = init_value if init_value is not None else 0.
        self._instance = qiskit_Parameter(name)
        self._value = init_value % self.MAX_VALUE
        self._name = name
    
    def __str__(self):
        ret = "{:5.3f}".format(self.value)
        return ret
    
    def __repr__(self):
        return self.__str__()
    
    def feed_dict(self, value: float = None):
        value = value if value is not None else self._value
        value = value % self.MAX_VALUE
        return {self.name: value}
    
    @property
    def value(self):
        return self._value
    
    @value.setter
    def value(self, val):
        val = val % self.MAX_VALUE
        self._value = val
    
    @property
    def instance(self):
        return self._instance
    
    @property
    def param(self):
        return self.instance
    
    @property
    def name(self):
        return self._name
    
    @classmethod
    def new_at_random(cls, name=None):
        val = np.random.rand() * cls.MAX_VALUE
        return cls(name, val)
    
    @classmethod
    def new_with_zero(cls, name=None):
        return cls(name, 0.)

In [25]:
param = Parameter.new_zero_param("param_0")
param

0.000

In [98]:
GateInfo = namedtuple('GateInfo', 'gate_class trainable multi_bit')
GATESET = [
    GateInfo(RXGate, True, False),
    GateInfo(RYGate, True, False),
    GateInfo(RZGate, True, False),
    GateInfo(CZGate, False, True),
]

In [99]:
class Gate:
    
    MULTI_BIT_GATES = [CZGate]
    
    def __init__(
            self,
            gate_info: GateInfo,
            qubit: int,
            parameter: Parameter,
    ):
        self._gate = gate_info.gate_class(parameter.instance)
        self._trainable: bool = gate_info.trainable
        self._multi_bit: bool = gate_info.multi_bit
        self._qubit: int = qubit
    
    @property
    def trainable(self):
        return self._trainable
    
    @property
    def multi_bit(self):
        return self._multi_bit
    
    @property
    def qubit(self):
        return self._qubit
    
    @property
    def gate(self):
        return self._gate

In [100]:
gate = Gate(gate_info=GATESET[0], qubit=0, parameter=Parameter("gate_1"))
gate

In [101]:
class Unit:
    
    def __init__(self, name, gates, parameters):
        self.name = name
        self._gates = gates
        self._params = parameters
    
    def feed_dict(self, values=None):
        if values is None:
            values = self.values
        if not hasattr(values, '__len__'):
            values = [values]
        assert len(self.parameters) == len(values)
        feed_dict = {}
        for param, value in zip(self.parameters, values):
            feed_dict |= param.feed_dict(value)
        return feed_dict
    
    def apply_to_circuit(self, qc: QuantumCircuit):
        for gate in self._gates:
            if gate.multi_bit:
                qargs = [
                    qc.qubits[gate.qubit[0]],
                    qc.qubits[gate.qubit[1]]
                ]
            else:
                qargs = [qc.qubits[gate.qubit]]
            qc.append(gate.gate, qargs=qargs)
        qc.barrier()
        return qc
    
    @property
    def values(self):
        return np.asarray([param.value for param in self.parameters])
    
    @values.setter
    def values(self, values):
        values = np.asarray(values)
        [
            param.value(value)
            for param, value in zip(self.parameters, values)
        ]
    
    @property
    def parameters(self):
        return self._params

In [103]:
nq = 3
ng = 3

gate_infos = [
    GATESET[g]
    for g in np.random.choice(len(GATESET), size=ng, replace=True)
]
print(gate_infos)

num_trainable = sum([gi.trainable for gi in gate_infos])
print(num_trainable)
parameters = [
    Parameter(f"unit_0_gate_{j}")
    for j, ginfo in enumerate(gate_infos)
    if ginfo.trainable
]
print(parameters)
# gates = [
#     Gate(GATESET[i], np.random.randint(nq), )
#     for i in np.random.choice(len(GATESET), size=3, replace=True)
# ]
# unit = Unit("unit", gates, parameters)

[GateInfo(gate_class=<class 'qiskit.circuit.library.standard_gates.ry.RYGate'>, trainable=True, multi_bit=False), GateInfo(gate_class=<class 'qiskit.circuit.library.standard_gates.rx.RXGate'>, trainable=True, multi_bit=False), GateInfo(gate_class=<class 'qiskit.circuit.library.standard_gates.rx.RXGate'>, trainable=True, multi_bit=False)]
3
[0.000, 0.000, 0.000]


In [73]:
unit_values = unit.values
unit.feed_dict(unit_values)

{'unit_0_gate_0': np.float64(0.0),
 'unit_0_gate_1': np.float64(0.0),
 'unit_0_gate_2': np.float64(0.0)}

In [74]:
qc = QuantumCircuit(nq)
unit.apply_to_circuit(qc)
qc.draw("mpl")

TypeError: issubclass() arg 1 must be a class